# Imports

In [81]:
import numpy as np

In [82]:
import matplotlib.pyplot as plt
import seaborn as sns

In [83]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input
from keras import Model

In [84]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

# Activation functions

In [ ]:
def identity(x):
    """Тождественная функция активации"""
    return x


def sigmoid(x):
    """Логистическая функция (сигмоида)"""
    return tf.math.sigmoid(x)


@tf.custom_gradient
def unit_step(x):
    """Единичная ступенька"""
    result = tf.cast(tf.math.greater_equal(x, 0), tf.float32)

    def grad(dy):
        return dy * sigmoid(x) * (1 - sigmoid(x))

    return result, grad


# def unit_step(x):
#     """Единичная ступенька"""
#     return np.greater_equal(x, 0)


def tanh_activation(x):
    """Гиперболический тангенс"""
    return tf.math.tanh(x)


def arctan_activation(x):
    """Арктангенс"""
    return tf.math.atanh(x)

In [ ]:
activation_functions = {
    "Identity": identity,
    "Unit step": unit_step,
    "Sigmoid": sigmoid,
    "Tanh": tanh_activation,
    "Arctan": arctan_activation,
}

# Data

In [86]:
def load_iris_binary():
    iris = load_iris()
    # Оставляем только 2 класса (0 и 1) для бинарной классификации
    X = iris.data[iris.target != 2]
    y = iris.target[iris.target != 2]
    return X, y

# Perceptron

In [87]:
class SingleLayerPerceptron(Model):
    def __init__(self, input_dim, activation_function):
        super(SingleLayerPerceptron, self).__init__()
        self.inputs = Input(shape=(input_dim,))
        self.dense = Dense(1, input_dim=input_dim)
        self.activation = activation_function

    def call(self, inputs):
        x = self.dense(inputs)
        return self.activation(x)

In [88]:
# Функция для обучения и оценки модели
def train_and_evaluate_perceptron(
    activation_function, activation_name, X_train, X_test, y_train, y_test
):
    print(f"\n=== Модель с функцией активации: {activation_name} ===")

    # Создание модели
    model = SingleLayerPerceptron(
        input_dim=X_train.shape[1], activation_function=activation_function
    )

    # Компиляция модели
    # Для бинарной классификации используем binary_crossentropy
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    # Обучение модели
    history = model.fit(
        X_train,
        y_train,
        epochs=100,
        batch_size=8,
        validation_data=(X_test, y_test),
        verbose=0,
    )

    # Предсказание
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int).flatten()

    # Оценка модели
    train_accuracy = history.history["accuracy"][-1]
    test_accuracy = accuracy_score(y_test, y_pred_binary)

    print(f"Точность на обучении: {train_accuracy:.4f}")
    print(f"Точность на тесте: {test_accuracy:.4f}")

    return history, y_pred_binary, model

# Train

In [89]:
def plot_comparison(results, y_test):
    # График точности во время обучения
    plt.figure(figsize=(15, 5))

    # График 1: Точность на обучении
    plt.subplot(1, 2, 1)
    for name, result in results.items():
        plt.plot(result["history"].history["accuracy"], label=name)
    plt.title("Точность на обучении")
    plt.xlabel("Эпоха")
    plt.ylabel("Точность")
    plt.legend()

    # График 2: Точность на валидации
    plt.subplot(1, 2, 2)
    for name, result in results.items():
        plt.plot(result["history"].history["val_accuracy"], label=name)
    plt.title("Точность на валидации")
    plt.xlabel("Эпоха")
    plt.ylabel("Точность")
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Сравнительная таблица точности
    print("\n=== СРАВНИТЕЛЬНАЯ ТАБЛИЦА ТОЧНОСТИ ===")
    accuracies = [(name, result["test_accuracy"]) for name, result in results.items()]
    accuracies.sort(key=lambda x: x[1], reverse=True)

    for name, acc in accuracies:
        print(f"{name}: {acc:.4f}")

    # Матрицы ошибок для лучшей модели
    best_model_name = accuracies[0][0]
    best_predictions = results[best_model_name]["predictions"]

    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(y_test, best_predictions)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Матрица ошибок - {best_model_name}\nТочность: {accuracies[0][1]:.4f}")
    plt.ylabel("Истинные классы")
    plt.xlabel("Предсказанные классы")
    plt.show()

In [ ]:
X, y = load_iris_binary()

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Масштабирование данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

results = {}

# Обучение моделей с разными функциями активации
for name, activation in activation_functions.items():
    history, y_pred, model = train_and_evaluate_perceptron(
        activation, name, X_train, X_test, y_train, y_test
    )
    results[name] = {
        "history": history,
        "predictions": y_pred,
        "model": model,
        "test_accuracy": accuracy_score(y_test, y_pred),
    }

# Визуализация результатов
plot_comparison(results, y_test)


=== Модель с функцией активации: Identity ===


d:\PyVenv\Neural\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Точность на обучении: 1.0000
Точность на тесте: 1.0000

=== Модель с функцией активации: Unit step ===


d:\PyVenv\Neural\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: Exception encountered when calling SingleLayerPerceptron.call().

[1m'SymbolicTensor' object has no attribute 'numpy'[0m

Arguments received by SingleLayerPerceptron.call():
  • inputs=tf.Tensor(shape=(None, 4), dtype=float32)

# Questions

## 1. Что делает функция активации в искусственном нейроне



## 2. Чем отличается искусственный нейрон от однослойного перцептрона?


## 3. Какое правило используется для обучения перцептрона?


## 4. Почему однослойный перцептрон не может решить задачу XOR?


## 5. Что произойдет, если изменить функцию активации с сигмоиды на ReLU?
